### Use season 2021-2022 (in Sep 2021) data to do testing

In [21]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [22]:
# df_nn_test.csv contains racing result in season 2021-2022 (Sep 5,2021 to Oct 1, 2021) as of Oct 3, 2021
df_nn_test=pd.read_csv('results_nn_test.csv')

In [23]:
df_nn_test.columns

Index(['r_season', 'r_date', 'r_index', 'r_course', 'r_class', 'r_distance',
       'r_tract', 'r_race_no', 'r_horse_no', 'r_brand_no', 'r_horse_age',
       'r_jockey', 'r_draw', 'r_rating', 'r_trainer', 'r_LBW',
       'r_finish_order', 'r_label', 'r_jockey_pct', 'r_trainer_pct', 'r_ct',
       'r_course_val', 'r_lot', 'r_lot_num', 'r_lot_seq', 'r_same_cd_1',
       'r_same_cd_2', 'r_same_cd_3', 'r_same_cd_4', 'r_same_cd_x',
       'r_sea_race_ind', 'r_draw_val', 'r_same_jh_1', 'r_same_jh_2',
       'r_same_jh_3', 'r_same_jh_4', 'r_same_jh_x', 'r_same_tj_1',
       'r_same_tj_2', 'r_same_tj_3', 'r_same_tj_4', 'r_same_tj_x'],
      dtype='object')

In [24]:
len(df_nn_test)  # <- Contains 72 races

849

In [25]:
df_race_index=df_nn_test.groupby(["r_index"])["r_index"].count()
print(df_race_index)    # <- Contains 72 of races

r_index
1     14
2      8
3     12
4     13
5     14
      ..
69     9
70    14
71    10
73    14
74    13
Name: r_index, Length: 72, dtype: int64


In [26]:
# Select target features
df_test=df_nn_test[['r_index',  'r_horse_no',   'r_class', 'r_draw_val',  'r_horse_age',
                 'r_rating',     'r_jockey_pct', 'r_trainer_pct',
                 'r_same_cd_1', 'r_same_cd_2', 'r_same_cd_3',  
                 'r_same_jh_1', 'r_same_jh_2', 'r_same_jh_3', 
                 'r_same_tj_1', 'r_same_tj_2', 'r_same_tj_3'
                 ##,'r_label']]
                 ]].sort_values(by=['r_index','r_horse_no'])

## Do predicton for season 2021-2022

In [40]:
# Load model
model = tf.keras.models.load_model( "Horse_Results")

## Change p_r_index to predict which race

In [50]:
# Select race (race index number for do prediction)
p_r_index=51

df_test_pred=df_test.loc[df_test['r_index']==p_r_index]

# Drop column 'r_index'
df_test_pred.drop(['r_index','r_horse_no' ], axis=1 , inplace=True)
df_test_pred.reset_index(drop=True, inplace=True)
df_test_pred.head(2)

/Users/pelecheung/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3990: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,r_class,r_draw_val,r_horse_age,r_rating,r_jockey_pct,r_trainer_pct,r_same_cd_1,r_same_cd_2,r_same_cd_3,r_same_jh_1,r_same_jh_2,r_same_jh_3,r_same_tj_1,r_same_tj_2,r_same_tj_3
0,4,5,5,60,5.31,9.38,0,0,0,0,0,0,9.28,3.09,4.12
1,4,2,5,59,4.24,9.68,0,0,0,0,0,0,5.26,7.89,13.16


In [51]:

# Start Predit
preds = model.predict(df_test_pred)
# print("preds shape:", preds.shape)
#print(preds)

# Rename predict result to 'Pred'
df_H_Preds=pd.DataFrame(preds)
df_H_Preds.columns = ['Pred']

# Add column 'Horse_no'
df_H_Preds.insert(loc=0, column='Horse_no', value=np.arange(len(df_H_Preds))+1)

# Sort the predict result
df_H_Preds.sort_values(by='Pred', inplace=True)

# Add new colum Pred_Position 
df_H_Preds.insert(loc=0, column='Pred_Position', value=np.arange(len(df_H_Preds))+1)

df_H_Preds['Race_Index'] = p_r_index

# Sort result based on Horse_no

df_HP_out=df_H_Preds[['Race_Index', 'Horse_no', 'Pred_Position']].sort_values(by='Horse_no').reset_index(drop=True)

print(df_HP_out.shape)

(12, 3)


In [52]:
# Check actual results with predict results
# Get actual from df_nn_test
df_nn_labels=df_nn_test.loc[df_nn_test['r_index']==p_r_index]

# Use column r_hors_no r_label
df_labels=df_nn_labels[['r_horse_no','r_finish_order']]

df_lables_v=df_labels[['r_horse_no','r_finish_order']].sort_values(by='r_horse_no').reset_index(drop=True)
           
# print(df_lables_v.shape)
# Comapre file
df_HP_out[['r_finish_order','r_horse_no']]=df_lables_v[['r_finish_order','r_horse_no']]
df_HP_out

,Race_Index,Horse_no,Pred_Position,r_finish_order,r_horse_no
0,51,1,12,11,1
1,51,2,11,5,2
2,51,3,1,2,3
3,51,4,2,9,4
4,51,5,3,1,5
5,51,6,5,7,6
6,51,7,6,10,7
7,51,8,8,8,8
8,51,9,10,6,9
9,51,10,4,4,10
